<a href="https://colab.research.google.com/github/descartesmbogning/Marburg-virus-research/blob/main/3_analysis_visibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.stats import norm


In [ ]:
# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

In [ ]:
from scipy import stats #the only import you actually need for this
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [ ]:
#example function that tests distribution then picks test accordingly
def test_sig(x, y):
    if stats.shapiro(x).pvalue >= 0.05 and stats.shapiro(y).pvalue >= 0.05: #if they are normally distriuted
        print("t-test")
        return stats.ttest_ind(x,y)
    else:
        print("mann-whitney")
        return stats.mannwhitneyu(x, y)

## import data

In [ ]:
data = pd.read_csv('D:\MARBURG VIRUS DISEASES\TABLES/data_fund_region_country_and_org_clean_dummies_col_replace_grp.csv')#.drop(['Addresses'], axis=1)
data = data.loc[data['funding_yes'] == 1]
data.head(2)

,Unnamed: 0,wos_ID,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,east asia and pacific,...,fulbright foundation - usa,world bank - multilateral,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year
1,1,wos:000921279800001,1,1,National Institutes of Health (NIH)(United Sta...,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,national institutes of health (nih) [u19ai1427...,hybrid,NaN,NaN,NaN,2023.0
2,2,wos:000909001200001,1,1,Russian Science Foundation(Russian Science Fou...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,russian science foundation; [22-13-00036],"gold, green accepted",NaN,3.0,3.0,2023.0


In [ ]:
data.count().head(10)

Unnamed: 0                 577
wos_ID                     577
funding_yes                577
open_access_yes            577
Funding Name Preferred     575
europe and central asia    125
north america              431
africa                       2
sub-saharan africa          20
east asia and pacific       78
dtype: int64

# The visibility of Marburg Virus Diseases publications (impact)

## 'Times Cited, All Databases'

### proportion all

In [ ]:
dataT = data.T

dataT['countries'] = dataT.index

tca_std_list = []
tca_median_list = []
tca_mean_list = []
tca_sum_list = []
for country in data.columns:
    tca_sum_list.append(data.dropna(subset=[country])['Times Cited, All Databases'].sum())
    tca_mean_list.append(data.dropna(subset=[country])['Times Cited, All Databases'].mean())

# rename countries names on columns
countries = list(dataT.index)
tca_sum  = tca_sum_list
tca_mean = tca_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'tca_sum': tca_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'tca_mean': tca_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['tca_count'] = dataT_concat['tca_sum']/dataT_concat['tca_mean']

#####
dataT_concat_all_tca = dataT_concat[['countries','tca_sum','tca_mean','tca_count']].set_index('countries')

# save data
dataT_concat_all_tca.to_csv(f'../TABLES/region_countries_org_tca_sum_mean_count.csv')
dataT_concat_all_tca

,tca_sum,tca_mean,tca_count
countries,,,
Unnamed: 0,22280.0,40.731261,547.0
wos_ID,22280.0,40.731261,547.0
funding_yes,22280.0,40.731261,547.0
open_access_yes,22280.0,40.731261,547.0
Funding Name Preferred,22264.0,40.851376,545.0
...,...,...,...
Open Access Designations,20894.0,43.620042,479.0
"Times Cited, All Databases",22280.0,40.731261,547.0
180 Day Usage Count,10068.0,47.046729,214.0


In [ ]:
len(tca_sum)

In [ ]:
len(tca_mean)

In [ ]:
vvvv

### proportion fund research

In [ ]:
data_fund = data.loc[data['funding_yes'] == 1]
data_fund.head(2)

In [ ]:
dataT = data_fund.T

dataT['countries'] = dataT.index

tca_std_list = []
tca_median_list = []
tca_mean_list = []
tca_sum_list = []
for country in data.columns:
    tca_sum_list.append(data_fund.dropna(subset=[country])['Times Cited, All Databases'].sum())
    tca_mean_list.append(data_fund.dropna(subset=[country])['Times Cited, All Databases'].mean())

# rename countries names on columns
countries = list(dataT.index)
tca_sum  = tca_sum_list
tca_mean = tca_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'fund_tca_sum': tca_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'fund_tca_mean': tca_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['fund_tca_count'] = dataT_concat['fund_tca_sum']/dataT_concat['fund_tca_mean']

#####*****************
dataT_concat_fund_tca = dataT_concat[['countries','fund_tca_sum','fund_tca_mean','fund_tca_count']].set_index('countries')

# save data
# dataT_concat_fund.to_csv(f'../TABLES/countries_fund_tca_sum_mean_count.csv')
dataT_concat_fund_tca

### proportion no fund research

In [ ]:
data_no_fund = data.loc[data['funding_yes'] == 0]
data_no_fund.head(2)

In [ ]:
dataT = data_no_fund.T

dataT['countries'] = dataT.index

tca_std_list = []
tca_median_list = []
tca_mean_list = []
tca_sum_list = []
for country in data.columns:
    tca_sum_list.append(data_no_fund.dropna(subset=[country])['Times Cited, All Databases'].sum())
    tca_mean_list.append(data_no_fund.dropna(subset=[country])['Times Cited, All Databases'].mean())

# rename countries names on columns
countries = list(dataT.index)
tca_sum  = tca_sum_list
tca_mean = tca_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'no_fund_tca_sum': tca_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'no_fund_tca_mean': tca_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['no_fund_tca_count'] = dataT_concat['no_fund_tca_sum']/dataT_concat['no_fund_tca_mean']

#####

dataT_concat_no_fund_tca = dataT_concat[['countries','no_fund_tca_sum','no_fund_tca_mean','no_fund_tca_count']].set_index('countries')

# save data
# dataT_concat_no_fund.to_csv(f'../TABLES/countries_no_fund_tca_sum_mean.csv')
dataT_concat_no_fund_tca

### concat and save data

In [ ]:
# concatenate dataframes on index
concat_data_pub_tca_list = pd.concat([dataT_concat_all_tca,
                                    dataT_concat_fund_tca,
                                    dataT_concat_no_fund_tca],
                                    axis=1)
# save data
concat_data_pub_tca_list.to_csv(f'../TABLES/concat_data_pub_tca_list.csv')
concat_data_pub_tca_list

### significance

In [ ]:
df_fund = data[['funding_yes','Times Cited, All Databases']]
df_fund

In [ ]:
df_fund.count()

In [ ]:
df_fund.reset_index().groupby(['funding_yes']).count()

In [ ]:
df_fund = df_fund.dropna()
df_fund

In [ ]:
df_fund.reset_index().groupby(['funding_yes']).count()

In [ ]:
df_fund.describe()

In [ ]:
df_fund.loc[df_fund['funding_yes'] == 0].describe()

In [ ]:
df_fund.loc[df_fund['funding_yes'] == 1].describe()

In [ ]:
#------------

In [ ]:
fund_TCA = df_fund.loc[df_fund['funding_yes'] == 1]['Times Cited, All Databases'].to_list()
fund_TCA[:15]

In [ ]:
no_fund_TCA = df_fund.loc[df_fund['funding_yes'] == 0]['Times Cited, All Databases'].to_list()

In [ ]:
test_sig(fund_TCA, no_fund_TCA)

In [ ]:
import plotly.express as px
fig = px.box(df_fund, x="funding_yes", y="Times Cited, All Databases", color="funding_yes")
# setting the y-axis range from 0 to 50
fig.update_layout(yaxis_range=[-2,100])
fig.update_layout(font_size=20)

fig.show()

## '180 Day Usage Count'

### proportion all

In [ ]:
dataT = data.T

dataT['countries'] = dataT.index

last_180_Day_Usage_Count_std_list = []
last_180_Day_Usage_Count_median_list = []
last_180_Day_Usage_Count_mean_list = []
last_180_Day_Usage_Count_sum_list = []
for country in data.columns:
    last_180_Day_Usage_Count_sum_list.append(data.dropna(subset=[country])['180 Day Usage Count'].sum())
    last_180_Day_Usage_Count_mean_list.append(data.dropna(subset=[country])['180 Day Usage Count'].mean())

# rename countries names on columns
countries = list(dataT.index)
last_180_Day_Usage_Count_sum  = last_180_Day_Usage_Count_sum_list
last_180_Day_Usage_Count_mean = last_180_Day_Usage_Count_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'all_180_Day_Usage_Count_sum': last_180_Day_Usage_Count_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'all_180_Day_Usage_Count_mean': last_180_Day_Usage_Count_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['all_180_Day_Usage_Count_count'] = dataT_concat['all_180_Day_Usage_Count_sum']/dataT_concat['all_180_Day_Usage_Count_mean']

#####


dataT_concat_all_180_day = dataT_concat[['countries','all_180_Day_Usage_Count_sum','all_180_Day_Usage_Count_mean']].set_index('countries')

# save data
# dataT_concat_all.to_csv(f'../TABLES/countries_all_180_Day_Usage_Count_sum_mean.csv')
dataT_concat_all_180_day

In [ ]:
len(last_180_Day_Usage_Count_sum)

In [ ]:
len(last_180_Day_Usage_Count_mean)

### proportion fund research

In [ ]:
data_fund = data.loc[data['funding_yes'] == 1]
data_fund.shape

In [ ]:
dataT = data_fund.T

dataT['countries'] = dataT.index

last_180_Day_Usage_Count_std_list = []
last_180_Day_Usage_Count_median_list = []
last_180_Day_Usage_Count_mean_list = []
last_180_Day_Usage_Count_sum_list = []
for country in data.columns:
    last_180_Day_Usage_Count_sum_list.append(data_fund.dropna(subset=[country])['180 Day Usage Count'].sum())
    last_180_Day_Usage_Count_mean_list.append(data_fund.dropna(subset=[country])['180 Day Usage Count'].mean())

# rename countries names on columns
countries = list(dataT.index)
last_180_Day_Usage_Count_sum  = last_180_Day_Usage_Count_sum_list
last_180_Day_Usage_Count_mean = last_180_Day_Usage_Count_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'fund_180_Day_Usage_Count_sum': last_180_Day_Usage_Count_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'fund_180_Day_Usage_Count_mean': last_180_Day_Usage_Count_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['fund_180_Day_Usage_Count_count'] = dataT_concat['fund_180_Day_Usage_Count_sum']/dataT_concat['fund_180_Day_Usage_Count_mean']

#####


dataT_concat_fund_180_day = dataT_concat[['countries','fund_180_Day_Usage_Count_sum','fund_180_Day_Usage_Count_mean','fund_180_Day_Usage_Count_count']].set_index('countries')

# save data
# dataT_concat_fund.to_csv(f'../TABLES/countries_fund_last_180_Day_Usage_Count_sum_mean.csv')
dataT_concat_fund_180_day

### proportion no fund research

In [ ]:
data_no_fund = data.loc[data['funding_yes'] == 0]
data_no_fund

In [ ]:
dataT = data_no_fund.T

dataT['countries'] = dataT.index

last_180_Day_Usage_Count_std_list = []
last_180_Day_Usage_Count_median_list = []
last_180_Day_Usage_Count_mean_list = []
last_180_Day_Usage_Count_sum_list = []
for country in data.columns:
    last_180_Day_Usage_Count_sum_list.append(data_no_fund.dropna(subset=[country])['180 Day Usage Count'].sum())
    last_180_Day_Usage_Count_mean_list.append(data_no_fund.dropna(subset=[country])['180 Day Usage Count'].mean())

# rename countries names on columns
countries = list(dataT.index)
last_180_Day_Usage_Count_sum  = last_180_Day_Usage_Count_sum_list
# tca_std = tca_std_list
# tca_median = tca_median_list
last_180_Day_Usage_Count_mean = last_180_Day_Usage_Count_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'no_fund_180_Day_Usage_Count_sum': last_180_Day_Usage_Count_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'no_fund_180_Day_Usage_Count_mean': last_180_Day_Usage_Count_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['no_fund_180_Day_Usage_Count_count'] = dataT_concat['no_fund_180_Day_Usage_Count_sum']/dataT_concat['no_fund_180_Day_Usage_Count_mean']

#####


dataT_concat_no_fund_180_day = dataT_concat[['countries','no_fund_180_Day_Usage_Count_sum','no_fund_180_Day_Usage_Count_mean','no_fund_180_Day_Usage_Count_count']].set_index('countries')

# save data
# dataT_concat_no_fund.to_csv(f'../TABLES/countries_no_fund_last_180_Day_Usage_Count_sum_mean_count.csv')
dataT_concat_no_fund_180_day

### concat and save data

In [ ]:
# concatenate dataframes on index
concat_data_pub_180_day_list = pd.concat([dataT_concat_all_180_day,
                                    dataT_concat_fund_180_day,
                                    dataT_concat_no_fund_180_day],
                                    axis=1)
# save data
concat_data_pub_180_day_list.to_csv(f'../TABLES/concat_data_pub_180_day_usage_list.csv')
concat_data_pub_180_day_list

### significance

In [ ]:
df_fund = data[['funding_yes','180 Day Usage Count']]
df_fund

In [ ]:
df_fund.count()

In [ ]:
df_fund.reset_index().groupby(['funding_yes']).count()

In [ ]:
df_fund = df_fund.dropna()
df_fund

In [ ]:
df_fund.reset_index().groupby(['funding_yes']).count()

In [ ]:
df_fund.describe()

In [ ]:
df_fund.loc[df_fund['funding_yes'] == 0].describe()

In [ ]:
df_fund.loc[df_fund['funding_yes'] == 1].describe()

In [ ]:
#------------

In [ ]:
fund_TCA = df_fund.loc[df_fund['funding_yes'] == 1]['180 Day Usage Count'].to_list()
fund_TCA[:15]

In [ ]:
no_fund_TCA = df_fund.loc[df_fund['funding_yes'] == 0]['180 Day Usage Count'].to_list()

In [ ]:
test_sig(fund_TCA, no_fund_TCA)

In [ ]:
import plotly.express as px
fig = px.box(df_fund, x="funding_yes", y="180 Day Usage Count", color="funding_yes")
# setting the y-axis range from 0 to 50
fig.update_layout(yaxis_range=[-2,100])
fig.update_layout(font_size=20)

fig.show()

## 'Since 2013 Usage Count'

### proportion all

In [ ]:
dataT = data.T

dataT['countries'] = dataT.index

Since_2013_Usage_Coun_std_list = []
Since_2013_Usage_Coun_median_list = []
Since_2013_Usage_Coun_mean_list = []
Since_2013_Usage_Coun_sum_list = []
for country in data.columns:
    Since_2013_Usage_Coun_sum_list.append(data.dropna(subset=[country])['Since 2013 Usage Count'].sum())
    Since_2013_Usage_Coun_mean_list.append(data.dropna(subset=[country])['Since 2013 Usage Count'].mean())

# rename countries names on columns
countries = list(dataT.index)
Since_2013_Usage_Coun_sum  = Since_2013_Usage_Coun_sum_list
Since_2013_Usage_Coun_mean = Since_2013_Usage_Coun_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'all_since_2013_Usage_Coun_sum': Since_2013_Usage_Coun_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'all_since_2013_Usage_Coun_mean': Since_2013_Usage_Coun_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['all_since_2013_Usage_Coun_count'] = dataT_concat['all_since_2013_Usage_Coun_sum']/dataT_concat['all_since_2013_Usage_Coun_mean']

#####


dataT_concat_all_since2013usage = dataT_concat[['countries','all_since_2013_Usage_Coun_sum','all_since_2013_Usage_Coun_mean','all_since_2013_Usage_Coun_count']].set_index('countries')

# save data
# dataT_concat_all.to_csv(f'../TABLES/countries_Since_2013_Usage_Count_sum_mean.csv')
dataT_concat_all_since2013usage

In [ ]:
len(Since_2013_Usage_Coun_sum)

In [ ]:
len(Since_2013_Usage_Coun_mean)

### proportion fund research

In [ ]:
data_fund = data.loc[data['funding_yes'] == 1]
data_fund.shape

In [ ]:
dataT = data_fund.T

dataT['countries'] = dataT.index

Since_2013_Usage_Coun_std_list = []
Since_2013_Usage_Coun_median_list = []
Since_2013_Usage_Coun_mean_list = []
Since_2013_Usage_Coun_sum_list = []
for country in data.columns:
    Since_2013_Usage_Coun_sum_list.append(data_fund.dropna(subset=[country])['Since 2013 Usage Count'].sum())
    Since_2013_Usage_Coun_mean_list.append(data_fund.dropna(subset=[country])['Since 2013 Usage Count'].mean())

# rename countries names on columns
countries = list(dataT.index)
Since_2013_Usage_Coun_sum  = Since_2013_Usage_Coun_sum_list
Since_2013_Usage_Coun_mean = Since_2013_Usage_Coun_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'fund_since_2013_Usage_Coun_sum': Since_2013_Usage_Coun_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'fund_since_2013_Usage_Coun_mean': Since_2013_Usage_Coun_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['fund_since_2013_Usage_Coun_count'] = dataT_concat['fund_since_2013_Usage_Coun_sum']/dataT_concat['fund_since_2013_Usage_Coun_mean']

#####


dataT_concat_fund_since2013usage = dataT_concat[['countries','fund_since_2013_Usage_Coun_sum','fund_since_2013_Usage_Coun_mean','fund_since_2013_Usage_Coun_count']].set_index('countries')

# save data
# dataT_concat_fund.to_csv(f'../TABLES/countries_fund_Since_2013_Usage_Count_sum_mean.csv')
dataT_concat_fund_since2013usage

### proportion no fund research

In [ ]:
data_no_fund = data.loc[data['funding_yes'] == 0]
data_no_fund.shape

In [ ]:
dataT = data_no_fund.T

dataT['countries'] = dataT.index

Since_2013_Usage_Coun_std_list = []
Since_2013_Usage_Coun_median_list = []
Since_2013_Usage_Coun_mean_list = []
Since_2013_Usage_Coun_sum_list = []
for country in data.columns:
    Since_2013_Usage_Coun_sum_list.append(data_no_fund.dropna(subset=[country])['Since 2013 Usage Count'].sum())
    Since_2013_Usage_Coun_mean_list.append(data_no_fund.dropna(subset=[country])['Since 2013 Usage Count'].mean())

# rename countries names on columns
countries = list(dataT.index)
Since_2013_Usage_Coun_sum  = Since_2013_Usage_Coun_sum_list
Since_2013_Usage_Coun_mean = Since_2013_Usage_Coun_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'no_fund_since_2013_Usage_Coun_sum': Since_2013_Usage_Coun_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'no_fund_since_2013_Usage_Coun_mean': Since_2013_Usage_Coun_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['no_fund_since_2013_Usage_Coun_count'] = dataT_concat['no_fund_since_2013_Usage_Coun_sum']/dataT_concat['no_fund_since_2013_Usage_Coun_mean']

#####


dataT_concat_no_fund_since2013usage = dataT_concat[['countries','no_fund_since_2013_Usage_Coun_sum','no_fund_since_2013_Usage_Coun_mean','no_fund_since_2013_Usage_Coun_count']].set_index('countries')

# save data
# dataT_concat_no_fund.to_csv(f'../TABLES/countries_no_fund_Since_2013_Usage_Count_sum_mean.csv')
dataT_concat_no_fund_since2013usage

### concat and save data

In [ ]:
# concatenate dataframes on index
concat_data_pub_tca_list = pd.concat([dataT_concat_all_since2013usage,
                                    dataT_concat_fund_since2013usage,
                                    dataT_concat_no_fund_since2013usage],
                                    axis=1)
# save data
concat_data_pub_tca_list.to_csv(f'../TABLES/concat_data_pub_since2013usage_list.csv')
concat_data_pub_tca_list

### significance

In [ ]:
df_fund = data[['funding_yes','Since 2013 Usage Count']]
df_fund

In [ ]:
df_fund.count()

In [ ]:
df_fund.reset_index().groupby(['funding_yes']).count()

In [ ]:
df_fund = df_fund.dropna()
df_fund

In [ ]:
df_fund.reset_index().groupby(['funding_yes']).count()

In [ ]:
df_fund.describe()

In [ ]:
df_fund.loc[df_fund['funding_yes'] == 0].describe()

In [ ]:
df_fund.loc[df_fund['funding_yes'] == 1].describe()

In [ ]:
#------------

In [ ]:
fund_TCA = df_fund.loc[df_fund['funding_yes'] == 1]['Since 2013 Usage Count'].to_list()
fund_TCA[:15]

In [ ]:
no_fund_TCA = df_fund.loc[df_fund['funding_yes'] == 0]['Since 2013 Usage Count'].to_list()

In [ ]:
test_sig(fund_TCA, no_fund_TCA)

In [ ]:
import plotly.express as px
fig = px.box(df_fund, x="funding_yes", y="Since 2013 Usage Count", color="funding_yes")
# setting the y-axis range from 0 to 50
fig.update_layout(yaxis_range=[-2,100])
fig.update_layout(font_size=20)

fig.show()